In [2]:
import pandas as pd

#### S3에서 데이터 불러오기

In [3]:
code = "https://ais7.s3.ap-northeast-2.amazonaws.com/%EC%82%AC%EB%B3%B8(29)+-+%ED%99%9C%EC%9A%A9%EB%8D%B0%EC%9D%B4%ED%84%B0/codeBook_v3.xlsx"
codeBook = pd.read_excel(code)

In [4]:
kp20 = "https://ais7.s3.ap-northeast-2.amazonaws.com/%EC%82%AC%EB%B3%B8(29)+-+%ED%99%9C%EC%9A%A9%EB%8D%B0%EC%9D%B4%ED%84%B0/KP2020.csv"
kp2020 = pd.read_csv(kp20, encoding='cp949', index_col=False)

In [5]:
kp21 = "https://ais7.s3.ap-northeast-2.amazonaws.com/%EC%82%AC%EB%B3%B8(29)+-+%ED%99%9C%EC%9A%A9%EB%8D%B0%EC%9D%B4%ED%84%B0/KP2021.csv"
kp2021 = pd.read_csv(kp21, encoding='cp949', index_col=False)

In [6]:
npa = "https://ais7.s3.ap-northeast-2.amazonaws.com/%EC%82%AC%EB%B3%B8(29)+-+%ED%99%9C%EC%9A%A9%EB%8D%B0%EC%9D%B4%ED%84%B0/NPA2020.csv"
npa2020 = pd.read_csv(npa, encoding='cp949', index_col=False)

#### concat

In [247]:
display(codeBook.loc[(codeBook.코드명 == "보이스피싱")])

,No,컬럼명,컬럼 그룹,코드명,코드값
57,58,EVT_CL_CD,사건종별코드,보이스피싱,215


In [248]:
# 보이스피싱 코드값에 해당하는 값만 추출
voice20 = kp2020.loc[(kp2020.EVT_CL_CD == 215)]
voice21 = kp2021.loc[(kp2021.EVT_CL_CD == 215)]
voice_npa20 = npa2020.loc[(npa2020.EVT_CL_CD == 215)]

In [249]:
print(f"voice20: {voice20.shape}, voice21: {voice21.shape}, voice_npa20: {voice_npa20.shape}")

voice20: (1660, 10), voice21: (33829, 10), voice_npa20: (10779, 10)


In [250]:
# kp2020, kp2021의 보이스피싱 컬럼만 먼저 concat
concat_voice = pd.concat([voice20, voice21])
# concat_voice.sample(10)

In [251]:
# RECV_CPLT_DM 20xx년 으로 변환을 위해 문자열앞에 20을 붙여줌
concat_voice["RECV_CPLT_DM"] = ["20" + concat_voice["RECV_CPLT_DM"].iloc[x] for x in range(len(concat_voice["RECV_CPLT_DM"]))]

# RECV_CPLT_DM 을 datetime 형변환
concat_voice["RECV_CPLT_DM"] = pd.to_datetime(concat_voice["RECV_CPLT_DM"])

# RECV_DEPT_NM 컬럼이 NPA_CL 컬럼값과 같아서 제거
concat_voice = concat_voice.drop(["RECV_DEPT_NM"], axis=1)

concat_voice.sample(5)

,RECV_CPLT_DM,NPA_CL,EVT_STAT_CD,EVT_CL_CD,RPTER_SEX,HPPN_PNU_ADDR,HPPN_X,HPPN_Y,SME_EVT_YN
1582582,2022-04-26 17:37:10,13,10,215,1.0,NaN,127.404663,36.341685,Y
1691158,2022-05-26 12:54:34,31,10,215,1.0,세종특별자치시 세종시 아름동 (행정:아름동 ),127.246219,36.511808,NaN
2571391,2023-01-10 15:54:15,13,10,215,1.0,NaN,127.404663,36.341685,NaN
446626,2021-05-24 18:34:19,13,10,215,2.0,대전광역시 대덕구 중리동(중리동) 275-30,127.421013,36.361819,NaN
1598247,2022-04-29 15:26:15,13,10,215,1.0,충청남도 계룡시 신도안면 정장리(신도안면 ) 273-4,127.231999,36.299860,N


In [252]:
# RECV_CPLT_DT, RECV_CPLT_TM 를 datetime 형변환을 위해 str로 형변환
# concat_voice와 콘캣시 타입에 맞추기위해 float로 형변환
voice_npa20 = voice_npa20.astype({"RECV_CPLT_DT" : "str", "RECV_CPLT_TM" : "str", "RPTER_SEX" : "float"})
voice_npa20.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10779 entries, 3221 to 1177944
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   RECV_CPLT_DT   10779 non-null  object 
 1   RECV_CPLT_TM   10779 non-null  object 
 2   NPA_CL         10779 non-null  int64  
 3   EVT_STAT_CD    10779 non-null  int64  
 4   EVT_CL_CD      10779 non-null  int64  
 5   RPTER_SEX      10779 non-null  float64
 6   HPPN_OLD_ADDR  10147 non-null  object 
 7   HPPN_X         10664 non-null  float64
 8   HPPN_Y         10664 non-null  float64
 9   SME_EVT_YN     6221 non-null   object 
dtypes: float64(3), int64(3), object(4)
memory usage: 926.3+ KB


In [253]:
# RECV_CPLT_TM 에 숫자가 6개가 아닐경우 0으로 채워줌
voice_npa20["RECV_CPLT_TM"] = [voice_npa20["RECV_CPLT_TM"].iloc[x].zfill(6) for x in range(len(voice_npa20["RECV_CPLT_TM"]))]

# RECV_CPLT_TM 시,분,초 단위로 나누기위해 슬라이싱을 통해 ':' 를 붙여줌
voice_npa20["RECV_CPLT_TM"] = [voice_npa20["RECV_CPLT_TM"].iloc[x][:2] + ":" + voice_npa20["RECV_CPLT_TM"].iloc[x][2:4] + ":" + voice_npa20["RECV_CPLT_TM"].iloc[x][4:] for x in range(len(voice_npa20["RECV_CPLT_TM"]))]

# 콘캣시키기 위해 RECV_CPLT_DM 컬럼을 생성하면서 RECV_CPLT_DT와 RECV_CPLT_TM를 합쳐줌
voice_npa20["RECV_CPLT_DM"] = voice_npa20["RECV_CPLT_DT"] + " " + voice_npa20["RECV_CPLT_TM"]

# RECV_CPLT_DM 을 datetime 형변환
voice_npa20["RECV_CPLT_DM"] = pd.to_datetime(voice_npa20["RECV_CPLT_DM"])

voice_npa20.sample(2)

,RECV_CPLT_DT,RECV_CPLT_TM,NPA_CL,EVT_STAT_CD,EVT_CL_CD,RPTER_SEX,HPPN_OLD_ADDR,HPPN_X,HPPN_Y,SME_EVT_YN,RECV_CPLT_DM
603464,20200726,16:36:15,13,10,215,2.0,대전광역시 서구 도마동(도마2동) 334-24,127.371118,36.316840,NaN,2020-07-26 16:36:15
29074,20200110,11:14:10,13,10,215,2.0,대전광역시 대덕구 송촌동(행정:송촌동) 227-1,127.441950,36.357615,NaN,2020-01-10 11:14:10


In [254]:
# RECV_CPLT_DM을 생성하였으므로 RECV_CPLT_DT, RECV_CPLT_TM 는 drop
voice_npa20 = voice_npa20.drop(["RECV_CPLT_DT", "RECV_CPLT_TM"], axis=1)

# 콘캣시 컬럼이름값이 같아야 해서 컬럼이름 변경
voice_npa20 = voice_npa20.rename(columns={'HPPN_OLD_ADDR' : 'HPPN_PNU_ADDR'})

# concat_voice의 컬럼순서와 맞춤
voice_npa20 = voice_npa20[['RECV_CPLT_DM', 'NPA_CL', 'EVT_STAT_CD', 'EVT_CL_CD',
       'RPTER_SEX', 'HPPN_PNU_ADDR', 'HPPN_X', 'HPPN_Y', 'SME_EVT_YN']]

voice_npa20.sample(2)

,RECV_CPLT_DM,NPA_CL,EVT_STAT_CD,EVT_CL_CD,RPTER_SEX,HPPN_PNU_ADDR,HPPN_X,HPPN_Y,SME_EVT_YN
266025,2020-03-18 14:01:25,13,10,215,1.0,대전광역시 서구 용문동(행정:용문동) 594-6,127.392509,36.338025,NaN
1166906,2020-11-19 01:04:18,19,10,215,3.0,충청남도 천안시 서북구 성정동(성정2동) 630-10,127.142265,36.825346,


In [255]:
# 머지이이
voice = pd.concat([concat_voice, voice_npa20])
# voice = merge_voice.merge(voice_npa20, how="left")
voice.sample(10)

,RECV_CPLT_DM,NPA_CL,EVT_STAT_CD,EVT_CL_CD,RPTER_SEX,HPPN_PNU_ADDR,HPPN_X,HPPN_Y,SME_EVT_YN
110383,2020-02-05 21:44:07,19,10,215,2.0,세종특별자치시 세종시 보람동(행정:보람동) 635-5,127.295141,36.480948,NaN
549751,2020-06-12 12:29:22,19,10,215,1.0,충청남도 아산시 온천동행정온양2동 2231,127.001072,36.781393,
1092391,2020-10-29 11:52:56,13,10,215,1.0,대전광역시 서구 괴정동(행정:괴정동) 13-2,127.380431,36.340856,Y
911013,2021-09-15 10:15:46,31,10,215,2.0,세종특별자치시 세종시 조치원읍 번암리(행정:조치원읍) 66-16,127.294908,36.588217,NaN
1560741,2022-04-12 21:44:56,19,10,215,2.0,충청남도 보령시 대천동 (행정:대천2동 ) 503-17,126.586165,36.358077,NaN
1136880,2020-11-11 14:02:38,19,10,215,2.0,충청남도 논산시 취암동(행정:취암동) 911,127.089458,36.194094,
2519248,2022-12-28 11:55:35,21,10,215,2.0,전라남도 담양군 대전면 대치리(행정:대전면 ) 870-4,126.889039,35.277085,NaN
928819,2020-09-17 11:36:19,19,10,215,2.0,세종특별자치시 세종시 아름동(행정:아름동) 1353,127.246790,36.510310,
1689161,2022-05-25 16:45:44,13,10,215,1.0,대전광역시 서구 갈마동 (갈마1동 ) 407,127.368924,36.343113,Y
2448002,2022-12-05 15:08:57,19,10,215,3.0,충청남도 예산군 삽교읍 상하리(행정:덕산면 ),126.673046,36.709144,Y


#### NPA_CL -> 코드명

In [230]:
# 코드명으로 변경을 위해 str로 형변환
voice = voice.astype({"NPA_CL" : "str", "EVT_STAT_CD" : "str", "RPTER_SEX" : "str"})

In [239]:
voice.NPA_CL.unique()

array(['충남청', '대전청', '서울청', '세종시', '인천청', '경기남부청', '전북청', '경남청', '1서울청',
       '광주청', '전남청', '부산청', '강원청', '대구청', '울산청', '경기북부청', '경북청'],
      dtype=object)

In [233]:
# NPA_CL 변경
voice["NPA_CL"] = voice["NPA_CL"].str.replace("19", "충남청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("13", "대전청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("8", "서울청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("31", "세종시")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("11", "인천청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("15", "경기남부청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("20", "전북청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("23", "경남청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("18", "충북청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("12", "광주청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("21", "전남청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("9", "부산청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("17", "강원청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("10", "대구청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("14", "울산청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("16", "경기북부청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("22", "경북청")

#### EVT_STAT_CD -> 코드명

In [240]:
voice.EVT_STAT_CD.unique()

array(['10', '5', '7', '9'], dtype=object)

In [241]:
voice["EVT_STAT_CD"] = voice["EVT_STAT_CD"].str.replace("10", "종결요청")
voice["EVT_STAT_CD"] = voice["EVT_STAT_CD"].str.replace("5", "도착")
voice["EVT_STAT_CD"] = voice["EVT_STAT_CD"].str.replace("7", "종결요청")
voice["EVT_STAT_CD"] = voice["EVT_STAT_CD"].str.replace("9", "다청이첩")

#### RPTER_SEX -> 코드명

In [242]:
voice.RPTER_SEX.unique()

array(['1.0', '2.0', '3.0'], dtype=object)

In [243]:
voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("1.0", "남성")
voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("2.0", "여성")
voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("3.0", "불상")

C:\Users\tpfl1\AppData\Local\Temp\ipykernel_8708\3265531194.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("1.0", "남성")
C:\Users\tpfl1\AppData\Local\Temp\ipykernel_8708\3265531194.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("2.0", "여성")
C:\Users\tpfl1\AppData\Local\Temp\ipykernel_8708\3265531194.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("3.0", "불상")


#### save

In [256]:
path = r"C:\Users\tpfl1\Desktop\solve_git\analysis"
voice.to_csv(f"{path}/voice.csv", index=False, encoding='cp949')